In [ ]:
"""
Author: Justin Perez (justin-perez@uiowa.edu)
Date: 2020-09-22

terminology:
    input length: count of 64-bit integers
    N: the set of input lengths to be tested
    n: represents a single value in N
    T: the set of '# of threads' values to be tested
    t: a single value in T
    mode: linux-style permission string, e.g., '-rw-rw-r--' (more info: https://www.comentum.com/images/permissions.jpg)
    

This notebook was tested with:
* hardware: 2020 M1 Macbook Pro (16 GB)
* python: 3.9.7 (distributed by homebrew)
    altair==4.1.0
    humanize==3.11.0
    jupyterlab==3.1.12
    pandas==1.3.3
* java: openjdk 16.0.2 (Temurin)
"""
from datetime import timedelta
from itertools import product
from pathlib import Path
import pandas as pd
import altair as alt
from stat import filemode
import humanize
from IPython.display import display


pd.options.display.max_colwidth = None

In [ ]:
DEBUG = False  # controls output verbosity. True -> more verbose; False -> less verbose
FORCE_CREATE_INPUTS = False  # forces recreation of test input files even if they already exist

T = [2**i for i in range(8)]
N = [10**i for i in range(4, 8)]
combinations = list(product(N, T))
iterations = 25 # how many times to execute each combo 


# chart/table labels
THREADS = 'T (# of threads)'
INPUT = 'N (# of 64-bit integers)'
REAL = 'real time (seconds)'
USER = 'user execution time (cpu seconds)'
SYS = 'system overhead time (cpu seconds)'
LABELS = [INPUT, THREADS, REAL, USER, SYS]

if DEBUG:
    print(f"{T=}")
    print(f"{N=}")
    print(f"{LABELS=}")
    humanized = [(humanize.intcomma(n), t) for n, t in combinations]
    display(pd.DataFrame(humanized, columns=[INPUT, THREADS]))

In [ ]:
# strict=True to fail fast if the directories dont exist
base_dir = Path('..').resolve(strict=True)
input_dir = base_dir / "tmp"
class_path = (base_dir/"out"/"production"/"ParallelSort").resolve(strict=True)
# it's ok if input/output don't exist yet
output_path = (base_dir / 'sorted.bin').resolve(strict=False)

# where to save the timing data from executing all those `combinations` `iterations` times,
# and whether to load an existing save file or collect a fresh set of data
timing_save_path = base_dir / 'observations.csv'
# True = load existing data from save path, False = make new data
skip_execution = load_existing_save = False
# force saving new data (only relevant when skip_execution/load_existing is False)
overwrite_existing_saved_timing = False

# figure out where java lives
java_bin = !which java
assert len(java_bin) == 1
java_bin = java_bin[0]

# command formatting
base_command = f"time {java_bin} {'-ea' if DEBUG else ''} -cp {class_path}"
sort_command = f"{base_command} hw1.ParallelExternalLongSorter"
generate_command = f"{base_command} hw1.DataFileGenerator"

if DEBUG:
    print(f"""
        {java_bin=}
        {class_path=}
        {base_dir=}
        {output_path=}
        {sort_command=}
        {generate_command=}
    """)

In [ ]:
def generate(filename: Path, length:int):
    output = !$generate_command $filename $length
    return output
        
def sort(input_path:Path, output_path:Path, threads:int=0):
    cmd = f"time {sort_command} {input_path} {output_path} {threads or ''}"
    output = !$cmd
    return output

def get_time(command_output):
    """super hacky util function for parsing output of time command
    probably won't work for non-zsh shells
    note: ipython uses your default shell unless specified otherwise
    
    returns a dictionary: {"real": seconds, "user": seconds, "sys": seconds}
        seconds: float
        real: the "wall clock time"
        user: time executing the command
        sys: time spent on system overhead
    """
    # find the right line
    grep = command_output.grep('^\s*([\d\.]*) r')
    # get rid of extra white space
    fields = grep.fields()
    # we got a list of fields for each line of output, but there should only be one line of output
    assert len(fields) == 1
    fields = fields[0]
    # we're going to turn this into a dictionary, but the fields are in reverse order
    fields = fields[::-1]
    # now we have something like ["user", "1.3", "real", "2.8", "sys", "3.2"] and need to turn it into pairs
    it = iter(fields)
    pairs = zip(it, it)
    return dict(pairs)

if DEBUG:
    input_path = (base_dir / 'data.bin').resolve(strict=False)
    debug_outputs = [
        generate(input_path, 100),
        sort(input_path, output_path),
        sort(input_path, output_path, 1),
        sort(input_path, output_path, 2),
    ]
    print(debug_outputs)
    assert input_path.resolve(strict=True)
    assert output_path.resolve(strict=True)

In [ ]:
input_paths = {
    n: (base_dir / str(n)).with_suffix('.bin')
    for n in N
}

for n, p in input_paths.items():
    if FORCE_CREATE_INPUTS or not p.exists():
        generate(p, n)
    
if DEBUG:
    columns = ['n', 'path', 'mode', 'size']
    rows = []
    for n, p in input_paths.items():
        assert p.exists()
        assert p.is_file()
        st = p.stat()
        rows.append([
            humanize.scientific(n, precision=0),
            str(p.resolve(strict=True)),
            filemode(st.st_mode),
            humanize.naturalsize(st.st_size),
        ])
    df = pd.DataFrame(rows, columns=columns)
    display(df)

In [ ]:
def get_sort_time(n, t):
    output = sort(
        input_path=input_paths[n],
        output_path=output_path,
        threads=t,
    )
    return (get_time(output), output)
    

if DEBUG:
    n, t = combinations[0]
    timing, output = get_sort_time(n ,t)
    print(f"{timing=}")

In [ ]:
timings = {
    combo: [] for combo in combinations
}

if DEBUG:
    outputs = {
        combo: [] for combo in combinations
    }

for combo in combinations:
    if skip_execution:
        break
    n, t = combo
    for i in range(iterations):
        if DEBUG:
            print(f"starting iteration #{i} sorting {n=} 64-bit integers on {t=} threads")
        timing, output = get_sort_time(n, t)
        timings[combo].append(timing)
        if DEBUG:
            outputs[combo].append(output)

In [ ]:
if load_existing_save:
    df = pd.read_csv(timing_save_path)
else:    
    # time to flatten everything out
    rows = []

    for (n, t), time_dicts in timings.items():
        for time_dict in time_dicts:
            rows.append([
                n,
                t,
                # Altair doesn't like timedeltas
                #timedelta(seconds=float(time_dict['real'])),
                #timedelta(seconds=float(time_dict['user'])),
                #timedelta(seconds=float(time_dict['sys' ])),
                float(time_dict['real']),
                float(time_dict['user']),
                float(time_dict['sys' ]),
            ])
    if DEBUG:
        print(rows[:5])
        print('...')
        print(rows[-5:])
        
    df = pd.DataFrame(rows, columns=LABELS)
    if not timing_save_path.exists() or overwrite_existing_saved_timing:
        df.to_csv(timing_save_path)

if DEBUG:
    display(df)
    display(df.dtypes)
    ls_timing = !ls -lh $timing_save_path  #| grep $timing_save_path
    print('\n'.join(ls_timing))

In [ ]:
chart_types = [REAL, USER, SYS]
charts = dict()
for chart_type in chart_types:
    line = alt.Chart(df).mark_line().encode(
      x=THREADS, #
      y=f'mean({chart_type}):Q',
      color=f'{INPUT}:N'
    )

    band = alt.Chart(df).mark_errorband(extent='ci').encode(
        x=THREADS,
        y=alt.Y(f'{chart_type}:Q'),
        color=f'{INPUT}:N'
    )

    charts[chart_type] = line + band

In [ ]:
charts[REAL]

In [ ]:
charts[USER]

In [ ]:
charts[SYS]